In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()



In [26]:


# hyperparameters
learning_rate = 1e-3
batch_size = 64
epochs = 5
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

import time

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    prev_time = time.time()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            cur_time = time.time()
            run_time = (cur_time-prev_time).__round__(3)
            prev_time = cur_time
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}] run_time(ms): {run_time}")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")



In [25]:

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.338010  [   64/60000] run_time: 10.0
loss: 0.482024  [ 6464/60000] run_time: 533.0
loss: 0.295489  [12864/60000] run_time: 463.0
loss: 0.518947  [19264/60000] run_time: 433.0
loss: 0.454077  [25664/60000] run_time: 430.0
loss: 0.456299  [32064/60000] run_time: 433.0
loss: 0.461853  [38464/60000] run_time: 454.0
loss: 0.639974  [44864/60000] run_time: 455.0
loss: 0.594365  [51264/60000] run_time: 470.0
loss: 0.406653  [57664/60000] run_time: 482.0
Test Error: 
 Accuracy: 83.0%, Avg loss: 0.476199 

Epoch 2
-------------------------------
loss: 0.335314  [   64/60000] run_time: 4.0
loss: 0.480296  [ 6464/60000] run_time: 449.0
loss: 0.293907  [12864/60000] run_time: 455.0
loss: 0.516957  [19264/60000] run_time: 442.0
loss: 0.451627  [25664/60000] run_time: 438.0
loss: 0.454327  [32064/60000] run_time: 474.0
loss: 0.459995  [38464/60000] run_time: 468.0
loss: 0.638145  [44864/60000] run_time: 471.0
loss: 0.592670  [51264/60000] run_time: 489

In [ ]:
import time
time.time().__round__(3)